<a href="https://colab.research.google.com/github/ChaitaliV/generative-explanation/blob/main/datacollection/unsupervised_data_collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Create Unsupervised Dataset for Generative Transformer
* read pdfs and save the data in .txt file.
* manually add some more data to it, from blogs and other online resources which are not downloadable.
* manually read the data once, and remove text which is not relevant
* clean the text
* create sentences
* create unsupervised learning dataset for generative transformer model by selectively masking words in all sentences

In [1]:
!pip install pyPDF2 transformers sentencepiece
!git clone https://github.com/ChaitaliV/generative-explanation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 71.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 90.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.1/311.1 kB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 117.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 40.8 MB/s eta 0:00:00
Cloning into 'generative-explanation'...
remote: Enumerating objects: 71, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 71 (delta 25), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (71/71), 4.23 MiB | 11.05 MiB/s, done.
Resolving deltas: 100% (25/25), done.


### Create Raw Dataset
* read text from the pdf which are textbooks, manuals and other web resources on depression and it's dignosis. this will be our raw data
* Go through the raw data manually, and add % to seperate topics, this should work better than seperating the text by sentences, as whole topics as input will retain long term dependencies.
* Also remove garbage text and topics that are not relevant.

In [2]:
from PyPDF2 import PdfReader
import os

In [3]:
folder_path = 'generative-explanation/datasets/unsupervised dataset'

In [4]:
def read_pdf(file_path):
    """fn to read pdf files"""
    with open(file_path, 'rb') as file:
        reader = PdfReader(file)
        text = ''
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            text += page.extract_text()
    return text

# get the list of all pdf from the folder
file_list = [file for file in os.listdir(folder_path) if file.endswith('.pdf')]
corpus = ''

# Iterate through PDF files and read text
for file_name in file_list:
    file_path = os.path.join(folder_path, file_name)
    text = read_pdf(file_path)
    corpus += text

#save the corpus as .txt file for future use
def save_to_txt(text, file_path):
    with open(file_path, 'w') as file:
        file.write(text)

save_to_txt(corpus, 'generative-explanation/raw_unsupervised_text.txt')

### Process the raw text to create Dataset with masked tokens
* load the raw data, remove numbers and special characters. and seperate the data from % token.
* mask words and create unsupervised dataset to predict the masked words.
* for masking process, mask individual masks

In [2]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import random
import numpy as np
import re
import pandas as pd

In [3]:
raw_data_path = 'generative-explanation/datasets/unsupervised dataset/raw_unsupervised_text.txt'

In [4]:
corpus = ''
with open(raw_data_path, 'r') as file:
  corpus = file.read()

In [18]:
tokenizer = T5Tokenizer.from_pretrained("t5-base")

def mask_tokens(sentence, mask_percentage=0.4):
    """"this function will take the sentence, generate tokens,
    randomly mask 40% tokens for encoder. unmasked 60% tokens are masked in decoder.
    """
    # Tokenize the sentence
    tokenized_sentence = tokenizer(sentence, return_tensors="pt",truncation=True,max_length = 512, pad_to_max_length=True, add_special_tokens=True).input_ids[0]
    label = tokenized_sentence.clone()

    # Calculate the number of tokens to mask
    num_tokens_to_mask = int(mask_percentage * len(tokenized_sentence))

    # Randomly choose indices to mask
    encoder_masked_indices = random.sample(range(1, len(tokenized_sentence) - 1), num_tokens_to_mask)
    decoder_masked_indices = list(set(np.arange(1, len(tokenized_sentence)-1)) - set(encoder_masked_indices))

    # Mask the chosen tokens in encoder
    for index in encoder_masked_indices:
        tokenized_sentence[index] = tokenizer.convert_tokens_to_ids(f"<masked_token>")


    #mask rest of the tokens in decoder
    for index in decoder_masked_indices:
        label[index] = tokenizer.convert_tokens_to_ids(f"<masked_token>")

    return tokenized_sentence, label

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:240: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [19]:
def generate_data(text):

  #remove new line characters
  text = text.replace('\n',' ')

  #remove special characters
  pattern = re.compile('[^a-zA-Z;?.,%\']')
  clean_text = pattern.sub(' ', text)
  clean_text = clean_text.replace('  ',' ')

  #create topic sequences
  text_strings = clean_text.split('%')

  #create masks for each topic strings
  encoder_data = []
  decoder_data = []
  for string in text_strings:
    encoder, decoder = mask_tokens(string)
    encoder_data.append(encoder)
    decoder_data.append(decoder)

  #create final dataframe
  df = pd.DataFrame({'Encoder': encoder_data, 'Decoder': decoder_data})
  return df

In [20]:
unsupervised_dataset = generate_data(corpus)
unsupervised_dataset

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


,Encoder,Decoder
0,"[tensor(23138), tensor(11), tensor(2), tensor(...","[tensor(23138), tensor(2), tensor(6261), tenso..."
1,"[tensor(389), tensor(2), tensor(27115), tensor...","[tensor(389), tensor(226), tensor(2), tensor(1..."
2,"[tensor(531), tensor(25), tensor(473), tensor(...","[tensor(531), tensor(2), tensor(2), tensor(2),..."
3,"[tensor(23138), tensor(3957), tensor(11), tens...","[tensor(23138), tensor(2), tensor(2), tensor(3..."
4,"[tensor(2502), tensor(15400), tensor(16), tens...","[tensor(2502), tensor(2), tensor(2), tensor(10..."
...,...,...
306,"[tensor(6632), tensor(7), tensor(2), tensor(2)...","[tensor(6632), tensor(2), tensor(13), tensor(1..."
307,"[tensor(555), tensor(2), tensor(2391), tensor(...","[tensor(555), tensor(16), tensor(2), tensor(2)..."
308,"[tensor(182), tensor(2), tensor(2), tensor(13)...","[tensor(182), tensor(422), tensor(381), tensor..."
309,"[tensor(3462), tensor(2), tensor(2), tensor(42...","[tensor(3462), tensor(3), tensor(9), tensor(2)..."


In [21]:
unsupervised_dataset.to_csv('unsupervised_dataset.csv')

So unsupervised dataset is ready with encoder and decoder data input tensors.